In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import sys

In [5]:
sys.path.insert(0, '../gahyparopt/')
sys.path.insert(0, '../evolution-und-ki')

In [6]:
from gahyperopt import *

In [25]:
import ipywidgets as widgets
import os, shutil, shlex
import json, io
from subprocess import Popen, PIPE, STDOUT
from contextlib import redirect_stdout, redirect_stderr
from datetime import datetime
import tensorflow as tf
import pandas
from matplotlib import pyplot
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from keras import backend as K
import glob
from collections import OrderedDict
import numpy

In [8]:
from GAUtilities import *

In [9]:
from parameters import *

In [10]:
def load_population():
    json_fnames = glob.glob("*.json")
    population = []
    names = []
    for j in json_fnames:
        name = j.split(".")[0]
        names.append(name)
        individuum = read_chromosome(j.split('.')[0])
        population.append(individuum)
        
    population = OrderedDict(zip(names, population))
    population=OrderedDict(sorted(population.items(),key=lambda x: x[1].accuracy, reverse=True))
    return population

In [11]:
POPULATION_SIZE=6

In [12]:
ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=POPULATION_SIZE,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        )

In [13]:
class model(object):
    generation_performance = {}
    population=None
    generations={0:None}

In [26]:
pull_button = widgets.Button(description="Generation aktualisieren")
def pull_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        git_pull(name='all')
pull_button.on_click(pull_button_clicked)
    
generation_count = widgets.BoundedIntText(value=0, min=0, max=0)

def on_generation_count_change(change):
    log_widget.clear_output()
    with log_widget:
        df = model.generation_performance[change['new']]
        
    plot_accuracy_vs_name(df)
generation_count.observe(on_generation_count_change, names=['value'])    

generation_button = widgets.Button(description="Generation laden")

def generation_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        model.population = load_population()

        df = pandas.DataFrame([
            {'name': key,
             'loss': model.population[key].loss,
             'accuracy': model.population[key].accuracy
            } for key in model.population.keys()
        ])
        generation_index = max(model.generations.keys())+1
        model.generation_performance[generation_index] = df
        model.generations[generation_index] = model.population
        generation_count.max = generation_index
        generation_count.min = 1
        generation_count.value = generation_index
    
generation_button.on_click(generation_button_clicked)

evolve_button = widgets.Button(description="Evolution!")
def evolve_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        new_chromosomes = ga.evolve_population(list(model.population.values()))
        model.population = OrderedDict(zip(model.population.keys(), new_chromosomes))
evolve_button.on_click(evolve_button_clicked)

distribute_button = widgets.Button(description="Generation verteilen")
def distribute_button_clicked(b):
    log_widget.clear_output()
    with log_widget:
        for name,individuum in model.population.items():
            write_chromosome(name=name,chromosome=individuum)
            git_push(name)
distribute_button.on_click(distribute_button_clicked)

accuracy_plot_widget = widgets.Output(layout={'border': '1px solid black', 'width':'50%', 'height':'500px', 'scroll':'true'})
# generation_tree_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})
log_widget = widgets.Output(layout={'border': '1px solid black', 'width':'80%', 'scroll':'true'})

def plot_accuracy_vs_name(df):
    accuracy_plot_widget.clear_output()
    with accuracy_plot_widget:
        fig, ax = pyplot.subplots(2,1,sharex=True,figsize=(7,8))
        df.iloc[::-1,::-1].plot.barh(x='name',y='accuracy', ax=ax[0],grid=True)
        df['accuracy'].plot.hist(ax=ax[1],grid=True, bins=numpy.linspace(0.,1.0, 21))
        show_inline_matplotlib_plots()

def plot_generation_tree():
    pass

def update_generation_tree(df):
    pass

gui = widgets.VBox(children=[
    widgets.HBox([pull_button, generation_button]),
    generation_count,
    accuracy_plot_widget,
#     generation_tree_widget,
    evolve_button,
    distribute_button,
    log_widget,
])

In [27]:
display(gui)

In [21]:
for pl, ind in model.population.items():
    print(pl, ind.parent_a, ind.parent_b, ind.id)
    #ind.ml_model.summary()

B hgwnpuljacaw czvlxeyrdbhw njxoeicgvmka
E hgwnpuljacaw wjmoexwllspq czvlxeyrdbhw
C czvlxeyrdbhw hgwnpuljacaw fnpokbmggqsf
F None None hgwnpuljacaw
D None None hgwnpuljacaw
A hgwnpuljacaw czvlxeyrdbhw vwraqzdepgru


In [29]:
import newick

In [32]:
# Generate tree
initial_population = model.generations[1]

In [36]:
winner = newick.Node.create(name=model.generations[-1], descendants=None)

In [38]:
for gen, pop in model.generations:
    
root.add_descendant(newick.Node(name='A'))

TypeError: add_descendant() missing 1 required positional argument: 'node'

In [48]:
model.generations[4]

OrderedDict([('B', <gahyperopt.Chromosome at 0x7f2e04767dc0>),
             ('E', <gahyperopt.Chromosome at 0x7f2e045494f0>),
             ('C', <gahyperopt.Chromosome at 0x7f2e045fbb80>),
             ('F', <gahyperopt.Chromosome at 0x7f2de16089a0>),
             ('D', <gahyperopt.Chromosome at 0x7f2de15bd280>),
             ('A', <gahyperopt.Chromosome at 0x7f2e04635700>)])

In [50]:
generation_count.value

0

In [52]:
winner = model.generations[4]['B']; winner

In [ ]:
for gen in range(4,0,-1):
    for parent in 
    winner_node.add_descendant(parent.id)

In [53]:
winner_node = newick.Node(name=winner.id)

In [ ]:
def add_parents(node, parent_gen):
    gen_dict = model.generations[parent_gen]
    chromosomes = list(gen_dict.values())
    
    parent_ids = 
    node.add_descendant()
    
    
    

In [43]:
for gen, pop in model.generations.items():
    print(gen)
    if gen>0:
        print(pop['D'].id)
          

0
1
hgwnpuljacaw
2
hgwnpuljacaw
3
hgwnpuljacaw
4
hgwnpuljacaw


In [31]:
model.generations

{0: None,
 1: OrderedDict([('D', <gahyperopt.Chromosome at 0x7f2e1c2a2d30>),
              ('C', <gahyperopt.Chromosome at 0x7f2e1c233c70>),
              ('F', <gahyperopt.Chromosome at 0x7f2e1dd3f340>),
              ('E', <gahyperopt.Chromosome at 0x7f2e1c16c040>),
              ('B', <gahyperopt.Chromosome at 0x7f2e1c1d4dc0>),
              ('A', <gahyperopt.Chromosome at 0x7f2e1c1f2fd0>)]),
 2: OrderedDict([('D', <gahyperopt.Chromosome at 0x7f2e142f6820>),
              ('C', <gahyperopt.Chromosome at 0x7f2e142b11f0>),
              ('A', <gahyperopt.Chromosome at 0x7f2e14265c70>),
              ('E', <gahyperopt.Chromosome at 0x7f2e14204eb0>),
              ('B', <gahyperopt.Chromosome at 0x7f2e14250a00>),
              ('F', <gahyperopt.Chromosome at 0x7f2e142e6670>)]),
 3: OrderedDict([('E', <gahyperopt.Chromosome at 0x7f2e04405340>),
              ('D', <gahyperopt.Chromosome at 0x7f2e044dccd0>),
              ('F', <gahyperopt.Chromosome at 0x7f2e044d2f40>),
              ('B